<a href="https://colab.research.google.com/github/cdans/FAQ-Recomennder-with-NLP/blob/main/TopicModelingFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model entwickeln

1. Anwenden eines Topic-Modeling-Models auf die vorhandenen Knowledgebase-Artikel, um die Topics dieser Artikel festzulegen
2. Nun soll ein neues Model trainiert werden, das neue Anfragen in die vorhandenen Topics einordnet.


In [ ]:
import os, re, operator, warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now

## Ausgangsdaten definieren

### Einlesen

Einlesen der CSV-Datei mit den Daten aus der Right-Now-Knowledgebase

In [ ]:
 import pandas as pd


In [ ]:

def read_data(file, columns = []):
  # in answers_v9 the data are seperated with ;;;. It could trigger import problems
  kb = pd.read_csv('' + file + '.csv', sep=';')
  #neu = pd.read_csv('answer_10373.csv', sep=',')
  
  # this will drop all rows with at least one NaN
  kb = kb.dropna(how='any')
  
  kb = kb.drop(columns=columns, axis=1)
  #kb = kb[:30]  
  return kb

# ***************************************************************
kb = read_data('answers_v9')
#kb = read_data('answers_v9', columns = [])
#cp = pd.read_csv('' + file + '.csv', sep=';')
#kb.iloc[228]
kb
id = kb['ID']
s = id.tolist()

for i in range(len(s)): 
  #print(i)
  z = re.sub('Answer ID ', '', s[i])
  s[i] = int(z)

v = kb.values.tolist()

kb = pd.DataFrame(v, index=s, columns=['Topic', 'ID', 'Question', 'Reference', 'Enviroment', 'Resolution' ])
#kb = kb.drop(columns='ID', axis=1)
#kb



neu = pd.read_csv('answer_10373.csv', sep=',')
l = []
for i in neu:
  l.append(i)
kb.loc[10373] = l
#kb[10373] = l
#kb = pd.DataFrame(neu, index=s, columns=['Topic', 'ID', 'Question', 'Reference', 'Enviroment', 'Resolution' ])
kb
#print(neu.values.tolist())
#zeile = zeile.drop(columns='ID', axis=1)
#zeile = neu.values.tolist()
#zeile

,Topic,ID,Question,Reference,Enviroment,Resolution
1,Notifying staff of new incidents,Answer ID 1,Can I receive an email when users have request...,"['Answer ID 7880', 'Answer ID 5168']",Staff Notifications,Incident Assigned Notification The Incident ...
9,Automatically assigning incidents to staff mem...,Answer ID 9,Is there a way of assigning incoming incidents...,"['Answer ID 2205', 'Answer ID 1041', 'Answer I...","Business Rules, All versions",The rules feature of Oracle B2C Service allow...
17,Sending incident responses to more than one em...,Answer ID 17,"Can I enter both the original contact, as well...",['Answer ID 1972'],"Incident response messages, All versions","When sending responses, the contact email add..."
31,Oracle B2C Service Infrastructure Requirements,Answer ID 31,What are the infrastructure requirements to ru...,"['Answer ID 11540', 'Answer ID 11390', 'Answer...","System Requirements Oracle B2C Service,Ã‚Â ...",Proper configuration and maintenance of your...
43,Number of files that can be attached to an inc...,Answer ID 43,How many files can I attach to incidents and a...,"['Answer ID 280', 'Answer ID 278', 'Answer ID ...",File Attachments Oracle B2C Service,The pages in Oracle B2C Service where files ...
...,...,...,...,...,...,...
7274,Order of resources inside move screen,Answer ID 7274,What order are the resources in inside of the ...,"['Answer ID 2543', 'Answer ID 9918', 'Answer I...","Oracle Field Service (OFS), Move Screen",When the move screen is activated the list o...
7305,Submitting an equipment/inventory related request,Answer ID 7305,What information should I include when submitt...,"['Answer ID 2543', 'Answer ID 9918', 'Answer I...",Equipment/inventory requests Oracle Field ...,When submitting an equipment/inventory relat...
7419,Incident Forwarding returns alert regarding 's...,Answer ID 7419,Why does one agent receive an error when forw...,['Answer ID 2056'],"Staff Accounts, Incidents - Outbound Emails...","When forwarding an incident, the system will..."
7445,Using PHP function file_get_contents,Answer ID 7445,Can I use the PHP function file_get_contents()...,['Answer ID 2056'],Customer Portal all versions,"Yes, as long as the URL you are passing is s..."


In [ ]:
kb = pd.read_csv('answers_v9.1.csv', sep=';')

# wirft alle Zeilen heraus, die mind. eine Zelle mit NaN haben
kb = kb.dropna(how='any')

kb

,Topic,ID,Question,Reference,Enviroment,Resolution
1,Notifying staff of new incidents,Answer ID 1,Can I receive an email when users have request...,"['Answer ID 7880', 'Answer ID 5168']",Staff Notifications,Incident Assigned Notification The Incident ...
3,Automatically assigning incidents to staff mem...,Answer ID 9,Is there a way of assigning incoming incidents...,"['Answer ID 2205', 'Answer ID 1041', 'Answer I...","Business Rules, All versions",The rules feature of Oracle B2C Service allow...
5,Sending incident responses to more than one em...,Answer ID 17,"Can I enter both the original contact, as well...",['Answer ID 1972'],"Incident response messages, All versions","When sending responses, the contact email add..."
7,Oracle B2C Service Infrastructure Requirements,Answer ID 31,What are the infrastructure requirements to ru...,"['Answer ID 11540', 'Answer ID 11390', 'Answer...","System Requirements Oracle B2C Service,Ã‚Â ...",Proper configuration and maintenance of your...
9,Number of files that can be attached to an inc...,Answer ID 43,How many files can I attach to incidents and a...,"['Answer ID 280', 'Answer ID 278', 'Answer ID ...",File Attachments Oracle B2C Service,The pages in Oracle B2C Service where files ...
...,...,...,...,...,...,...
3139,Order of resources inside move screen,Answer ID 7274,What order are the resources in inside of the ...,"['Answer ID 2543', 'Answer ID 9918', 'Answer I...","Oracle Field Service (OFS), Move Screen",When the move screen is activated the list o...
3141,Submitting an equipment/inventory related request,Answer ID 7305,What information should I include when submitt...,"['Answer ID 2543', 'Answer ID 9918', 'Answer I...",Equipment/inventory requests Oracle Field ...,When submitting an equipment/inventory relat...
3143,Incident Forwarding returns alert regarding 's...,Answer ID 7419,Why does one agent receive an error when forw...,['Answer ID 2056'],"Staff Accounts, Incidents - Outbound Emails...","When forwarding an incident, the system will..."
3145,Using PHP function file_get_contents,Answer ID 7445,Can I use the PHP function file_get_contents()...,['Answer ID 2056'],Customer Portal all versions,"Yes, as long as the URL you are passing is s..."


### Bereinigung
Imports 



In [ ]:
import numpy as np
import pandas as pd

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

import re
import string

#### Pipeline auf Resolution

Daten mit einer spaCy Pipeline aufbereiten



In [ ]:
nlp = spacy.load("en_core_web_sm")

# This takes in a doc of tokens from the NER and lemmatizes them. 
# Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
def lemmatizer(doc):
    doc = [token.lemma_ for token in doc if token.lemma_ != 'PRON']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)

# Removes all numbers without letters
def notNUM(doc):
  doc = [token.text for token in doc if token.pos_ != 'NUM']
  doc = u' '.join(doc)
  return nlp.make_doc(doc)

# Removes all symbols
def noSYM(doc):
  doc = [token.text.lower() for token in doc if token.pos_ != 'SYM']
  doc = u' '.join(doc)
  return nlp.make_doc(doc)
  
# Removes single characters
def single_char(doc):
  doc = [token.text for token in doc if len(token.text) > 2]
  doc = u' '.join(doc)
  return nlp.make_doc(doc)

# Removes punctuation
def rm_punct(doc):
  doc = [token.text for token in doc if not token.is_punct]
  doc = u' '.join(doc)
  return nlp.make_doc(doc)

# Removes adjectives
def rm_adj(doc):
  doc = [token.text for token in doc if token.pos_ != 'ADJ']
  doc = u' '.join(doc)
  return nlp.make_doc(doc)

# Removes other not identifable word
def rm_other(doc):
  doc = [token.text for token in doc if token.pos_ != 'X']
  doc = u' '.join(doc)
  return nlp.make_doc(doc)

# Removes verbs
def rm_verb(doc):
  doc = [token.text for token in doc if token.pos_ != 'VERB']
  doc = u' '.join(doc)
  return nlp.make_doc(doc)

Stop Words

In [ ]:

# Updates spaCy's default stop words list with my additional words
def add_own_stopwords(stop_list = []):
  nlp = spacy.load("en_core_web_sm")
  nlp.Defaults.stop_words.update(stop_list)
  
  # Iterates over the words in the stop words list and resets the "is_stop" flag.
  for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True


def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True and token.is_space != True]
    return doc


# *********************************************************************************

In [ ]:
def clean_column(column, my_stop_words, is_set = True):

  add_own_stopwords(my_stop_words)

  # For further working we will use the column "Resolution"
  cleankb = column

  nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
  nlp.add_pipe(notNUM,name='notNUM',after='ner')
  nlp.add_pipe(single_char,name='single_char',after='ner')
  nlp.add_pipe(noSYM,name='noSYM',after='ner')
  nlp.add_pipe(rm_punct,name='rm_punct',after='ner')
  nlp.add_pipe(rm_adj,name='rm_adj',after='ner')
  nlp.add_pipe(rm_other,name='rm_other',after='ner')
  #nlp.add_pipe(rm_verb,name='rm_verb',after='ner')
  nlp.add_pipe(remove_stopwords, name="stopwords", last=True) 


  liste = []
  index = 0
#  print(len(column))
  # tqdm = Fortschrittsbalken
  i = kb['Resolution'].index.tolist()

  for z in tqdm(column):
    z = re.sub('ã*', '', z)
    z = re.sub('\n', '', z)
    z = re.sub('â*', '', z)
    z = re.sub('ã*', '', z)
    z = re.sub('ë*', '', z)
    z = re.sub('\d*', '', z)
    #z = re.sub('*', '', z)
    z = re.sub('¢*', '', z)
    article_list = []
    pr = nlp(z)
    if is_set:
      liste.append(list(set(pr)))
      article_list.append(pr)
      cleankb[i[index]] = list(set(article_list[0]))
    else:
      liste.append(pr)
      article_list.append(pr)
      cleankb[i[index]] = article_list[0]
    
    pr = nlp(z)
    
    #liste.append(pr)  

    index += 1

  return liste, cleankb
  
    
    
  

# ****************************************************************************
# Define own Stop Words
my_stop_words = ['\ufeff1', '¢‚¬*', 'ë*']
column = kb['Resolution'].copy(deep=True)

# Ohne Duplikate
#doc_list, cleankb_resolution = clean_column(column, my_stop_words)

# Mit Duplikate
doc_list, cleankb_resolution = clean_column(column, my_stop_words, is_set = False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Speicherung der doc_list (Liste mit Strings)

In [ ]:
doc_csv = pd.DataFrame(doc_list)
doc_csv.to_csv('doc_list.csv', index=False)

Test / Anzeige von doc_list mit richtigen Indizes


In [ ]:
#for i in doc_list:
#  print(i)

ind = 0
for i in cleankb_resolution.index.tolist():
  print(i, doc_list[ind])
  ind +=1

1 ['incident', 'assign', 'notification', 'incident', 'assign', 'notification', 'notify', 'staff', 'member', 'new', 'incident', 'assign', 'incident', 'assign', 'message', 'notification', 'display', 'notification', 'section', 'communication', 'center', 'staff', 'member', 'account', 'configure', 'send', 'email', 'notification', 'staff', 'member', 'staff', 'member', 'log', 'oracle', 'service', 'staff', 'member', 'log', 'incident', 'assign', 'staff', 'member', 'receive', 'email', 'inbox', 'inform', 'notification', 'send', 'log', 'determine', 'action', 'need', 'regard', 'incident', 'configure', 'staff', 'account', 'send', 'email', 'notification', 'inbox', 'staff', 'member', 'log', 'oracle', 'service', 'use', 'step', 'click', 'configuration', 'button', 'select', 'staff', 'management', 'open', 'staff', 'account', 'group', 'staff', 'account', 'manager', 'staff', 'account', 'profile', 'report', 'double', 'click', 'staff', 'account', 'like', 'edit', 'staff', 'account', 'detail', 'select', 'approp

Test / Anzeige von cleankb_resolution

In [ ]:
cleankb_resolution

1       [incident, assign, notification, incident, ass...
3       [rule, feature, oracle, service, allow, assign...
5       [send, response, contact, email, address, fiel...
7       [proper, configuration, maintenance, environme...
9       [page, oracle, service, file, attach, associat...
                              ...                        
3139    [screen, activate, list, suggest, resource, fo...
3141    [submit, equipment, inventory, relate, request...
3143    [forward, incident, system, automatically, set...
3145    [yes, long, url, pass, setup, allow, http, nat...
3147    [access, configuration, assistant, self, servi...
Name: Resolution, Length: 1550, dtype: object

## Text Clustering

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score  

Creates a List with Strings

In [ ]:
clean_resu = cleankb_resolution.copy()
documents = []
for i in clean_resu.index:
  documents.append(" ".join(clean_resu[i]))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

# Prepare the coloumn Resolution for clustering
vectorizer = TfidfVectorizer(stop_words='english')
resolution_vec = vectorizer.fit_transform(documents)

In [ ]:
# Amount of clusters
true_k = 250

# Defines the model for clustering 
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=1000, n_init=1)
model.fit(resolution_vec)


# all words (as index of this word) will be assigned in a cluster
order_centroids = model.cluster_centers_.argsort()[:, ::-1]

# List of all terms who are present in row 'Resolution'
terms = vectorizer.get_feature_names()


# Print all Clusters
# !!!!! Speicherung in ein DataFrame, Dictonary etc. evtl sinnvoll
for i in range(true_k):
    temp = []
    for ind in order_centroids[i, :100]:
      temp.append(terms[ind])
    print("Cluster %d:" % i, temp)

Cluster 0: ['internet', 'zone', 'security', 'trust', 'site', 'click', 'level', 'press', 'set', 'explorer', 'adjust', 'medium', 'navigate', 'website', 'option', 'enable', 'tool', 'session', 'add', 'button', 'sign', 'local', 'setting', 'domain', 'low', 'tab', 'privilege', 'web', 'authenticode', 'select', 'apply', 'link', 'little', 'resolution', 'open', 'custom', 'file', 'https', 'browser', 'upload', 'directory', 'slider', 'page', 'server', 'high', 'restrict', 'belong', 'change', 'os', 'close', 'dependent', 'localstorage', 'lose', 'atg', 'content', 'com', 'uploader', 'script', 'rightnow', 'brokenlink', 'active', 'path', 'recommend', 'guide', 'configuration', 'application', 'initiate', 'example', 'panel', 'transition', 'like', 'question', 'run', 'box', 'reside', 'attach', 'allow', 'default', 'installation', 'prompt', 'include', 'oracle', 'work', 'intranet', 'tester', 'support', 'different', 'locate', 'fail', 'control', 'drop', 'able', 'method', 'disrupt', 'misconfiguration', 'icb', 'sure',

### Clustern zuweisen (Resolution) 

In [ ]:
print(kb['Resolution'].index.tolist())

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223, 225, 227, 229, 231, 233, 235, 237, 239, 241, 243, 245, 247, 249, 251, 253, 255, 257, 259, 261, 263, 265, 267, 269, 271, 273, 275, 277, 279, 281, 283, 285, 287, 289, 291, 293, 295, 297, 299, 301, 303, 305, 307, 309, 311, 313, 315, 317, 319, 321, 323, 325, 327, 329, 331, 333, 335, 337, 339, 341, 343, 345, 347, 349, 351, 353, 355, 357, 359, 361, 363, 365, 367, 369, 371, 373, 375, 377, 379, 381, 383, 385, 387, 389, 391, 393, 395, 397, 399, 401, 403, 405, 407, 409, 411, 413, 415, 417, 419, 421,

DataFrame mit Index, Text, Cluster 

model = 250 Cluster

cluster_results.csv

In [ ]:
cluster_results = []#{'Resolution': [], 'Cluster': []}

indizes = kb['Resolution'].index.tolist()
index = 0
# vectorize each cell in Resolution and make a prediction for a cluster
for i in cleankb_resolution:
  temp = ' '.join(i)
  vec = vectorizer.transform([temp])
  prediction = model.predict(vec)

  # saves the result in the dictonary cluster_results
  cluster_results.append([i,int(prediction)])
  #cluster_results['Cluster'].append(int(prediction))
  index += 1

# Transform the dictonary in a dataframe
cluster_view = pd.DataFrame(cluster_results, index = kb['Resolution'].index.tolist(), columns=['Text', 'Cluster'])

# cluster_view will save to csv format
cluster_view.to_csv('cluster_results_1.csv', index=kb['Resolution'].index.tolist())

# shows cluster_view
cluster_view

,Text,Cluster
1,"[incident, assign, notification, incident, ass...",32
3,"[rule, feature, oracle, service, allow, assign...",108
5,"[send, response, contact, email, address, fiel...",8
7,"[proper, configuration, maintenance, environme...",72
9,"[page, oracle, service, file, attach, associat...",12
...,...,...
3139,"[screen, activate, list, suggest, resource, fo...",204
3141,"[submit, equipment, inventory, relate, request...",204
3143,"[forward, incident, system, automatically, set...",47
3145,"[yes, long, url, pass, setup, allow, http, nat...",101


### Test

Bestimmt für eine bestimmte Anzahl an Cluster, wieviele Questions richtig oder falsch zugeordnet wurden

In [ ]:
def proof_prediction(quest, data, ldamodel):
  richtige_index = []
  right_cl = 0 #number of right matches
  wrong_cl = 0 #number of wrong matches

  indizes = data.index.tolist()
  ind = 0

  for i in tqdm(quest.index): # i = index of quest
    quest_index = i

    # Preparing the question 
    inc1 = nlp(quest[i])
    inc1 = ' '.join(inc1)
    incident = ([inc1])

    # Prediction
    incidentvec = vectorizer.transform(incident)
    prediction1 = ldamodel.predict(incidentvec) # Number of cluster
    
    # Cluster of Resolution
    c1 = data.Cluster.loc[i]
    
    # Cluster kb == cluster incident: right else: wrong
    if c1 == prediction1[0]:
      right_cl += 1
      richtige_index.append(indizes[ind])      
    else: 
      wrong_cl += 1
      #print(indizes[ind])
    ind += 1
  
  print('Richtig: ' + str(right_cl))
  print('Falsch: ' + str(wrong_cl))
  
  quote = 0
  if wrong_cl != 0:
    quote = round(right_cl / index, 4)
    print('Quote: ' + str(round(right_cl / index, 4)) + '%')
  return richtige_index, quote


# ********************************************
#clusterres = pd.read_csv('cluster_results.csv')

questions = kb['Question']

richtig_index, quot = proof_prediction(questions, cluster_view, model)

print(richtig_index)
print(quot)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



Richtig: 769
Falsch: 781
Quote: 0.4961%
[11, 17, 19, 23, 25, 27, 33, 35, 37, 41, 45, 47, 49, 51, 53, 63, 69, 71, 73, 77, 79, 81, 89, 91, 93, 95, 97, 107, 109, 111, 119, 121, 125, 129, 131, 133, 135, 137, 143, 145, 155, 163, 165, 167, 169, 177, 179, 181, 183, 187, 203, 209, 219, 225, 227, 233, 239, 245, 255, 257, 263, 265, 273, 277, 281, 287, 289, 291, 295, 297, 301, 313, 317, 323, 327, 329, 335, 337, 345, 347, 353, 357, 361, 363, 371, 373, 377, 383, 385, 393, 397, 399, 401, 403, 405, 407, 411, 415, 421, 425, 427, 429, 431, 437, 439, 443, 445, 447, 455, 469, 471, 477, 481, 485, 487, 495, 497, 499, 501, 503, 509, 517, 523, 535, 537, 539, 541, 543, 552, 556, 558, 566, 568, 574, 578, 580, 584, 590, 600, 606, 608, 612, 616, 624, 628, 630, 638, 648, 650, 652, 656, 662, 670, 674, 676, 682, 684, 686, 690, 694, 698, 700, 710, 712, 718, 722, 726, 728, 730, 732, 736, 738, 742, 744, 746, 748, 752, 754, 756, 758, 760, 766, 770, 782, 788, 792, 794, 800, 816, 824, 826, 828, 830, 832, 834, 836, 842, 

Iteriert durch eine Anzahl an Cluster und bestimmt jeweils die Anzahl der richtigen und falschen Zuordnungen.
Am Ende wird die beste Lösung angezeigt

In [ ]:
def proof_prediction1(quest):

  K = range(10,100, 2) # Number of Clusters
  max_quote = 0
  max_k = 0
  csv_result = []
  
  # Model for k Cluster
  for k in tqdm(K):
    index__df = []
    cluster_results = [] 
    km = KMeans(n_clusters=k, init='k-means++', max_iter=1000, n_init=1)
    km = km.fit(resolution_vec)

    # Assign Cluster in Resolution 
    for i in cleankb_resolution.index:
      temp = ' '.join(cleankb_resolution[i])
      vec = vectorizer.transform([temp])
      prediction = km.predict(vec)

      # saves the result in a list cluster_results
      index__df.append(i)
      cluster_results.append([
                              cleankb_resolution[i], 
                              int(prediction)
                              ])
    
    # Transform the cluster_results in a dataframe with original indizes
    cluster_view = pd.DataFrame(cluster_results, index = index__df, columns=['Text', 'Cluster'])
    
    # Returns a list of kb with same cluster and quote of right assignments
    richtige_indizes, quote = proof_prediction(questions, cluster_view, km)
    
    csv_result.append([k, quote])
    print(k)
    
    # Identifies the hightes quote
    if quote > max_quote:
      max_quote = quote
      max_k = k
  

  ###
  #inde = cleankb_resolution.index.tolist()
  ###
  df = pd.DataFrame(csv_result, columns=['k', 'quote'])
  df.to_csv('csv_result.csv', index=False)
  
  return max_quote, max_k




# ***************************************************************************
questions = kb['Question']

#quot, cluster = proof_prediction1(questions)


#print(quot)
#print('Opt K: ' + str(cluster))

In [ ]:
result = pd.read_csv('csv_result.csv')
result

In [ ]:
def test_cluster(incident):
  # Identify the index of the question
  index = 0
  index_cluster = []
  indizes = kb['Question'].index.tolist()
  
  for i in kb['Question']:
    if i == incident[0]:
      index_cluster.append(indizes[index])
    index += 1

  # Predict a cluster
  incidentvec = vectorizer.transform(incident)
  prediction1 = model.predict(incidentvec)
  
  # print results
  print('Incident: ', incident[0]) # String
  print('Cluster: ', prediction1) # Cluster of Incident
  

  if len(index_cluster) > 0:  # Cluster of his resolution if exist
    print('Cluster of Resolution: ', cluster_view.Cluster[index_cluster[0]])
  else:
    print("Not in Resolution")
  print('index_cluster: ', index_cluster) #index of the incident in kb

In [ ]:
# ******************************************
test_cluster(["Can I receive an email when users have requested help and an incident is created?"])

Incident:  Can I receive an email when users have requested help and an incident is created?
Cluster:  [47]
Cluster of Resolution:  32
index_cluster:  [1]


In [ ]:
test_cluster(["How many files can I attach to incidents and answers in Oracle B2C Service?"])

Incident:  How many files can I attach to incidents and answers in Oracle B2C Service?
Cluster:  [134]
Cluster of Resolution:  12
index_cluster:  [9]


## Test

In [ ]:
def test_sim_incidents(incident1, incident2):
  inc1 = nlp(incident1)
  inc1 = ' '.join(inc1)
  incident = ([inc1])
  
  incidentvec = vectorizer.transform(incident)
  prediction1 = model.predict(incidentvec)
  print(incident)
  print('Cluster: ', prediction1[0])
  
  inc2 = nlp(incident2)
  inc2 = ' '.join(inc2)
  incident2 = ([inc2])
  
  incidentvec2 = vectorizer.transform(incident2)
  prediction2 = model.predict(incidentvec2)
  print(incident2)
  print('Cluster: ', prediction2[0])

  #t = []
  #t.append(inc1)
  #vector_row = np.array(t)
  #print(vector_row)
  #inc2 = nlp(incident2)
  
  #inc1vec = vectorizer.transform(vector_row)
  #print(inc1vec)
  #inc2vec = vectorizer.transform(inc2)

  #prediction1 = model.predict(inc1vec)
  #prediction2 = model.predict(inc2vec)

  
  #pprint('Cluster: ', prediction1[0])
  #pprint('Cluster: ', prediction1[0])

In [ ]:
i1 = "How can I enable and customize the Support Home page?"
i2 = " Within the Customer Portal (CP) end-user pages, the home.php  Â file defines the functionality of the Support Home page. Staff whose profile allows access to the Customer Portal Studio can edit and customize Â the home.php file. This includes adding or removing widgets, text and features from the file. The path to access the home.php Â file is: euf/development/views/pages. The layout of the page is defined by the stylesheets used with your CP pages, which are accessed from the views/templates directory. To disable the Support Home page in the CP pages: Â  You must remove reference to the home.php file from other files and templates. In addition, you must set the default Home page to be another page in your CP pages. Remove references to the Home page. Â  Â The default CP files include a tab for the Home page in the navigation bar. Â  Â By default, the buttons across the top of the page is defined in the rn_standard.php and the rn_standard_account.php Â templates. Â  Â Edit the templates to remove reference to the home.php file. Â   Â  If you have customized other pages to link to the Home page, you must edit those files as well to remove the link to the home.php file.  Â  Set the default end-user page to be another page. Â  Â Set the CP_HOME_URL Â configuration setting to another of your CP pages. Â  Â Some widgets include a redirect function to Home page. Â  Â  Â By setting the default home page to another file, the user is directed to the page you define. For example, you can set the CP_HOME_URL setting to answers/list so that when users log in, they are directed to the Answers List page by default. The access this setting, select Configuration from the navigation area &gt; Site Configuration &gt; Configuration Settings &gt; search by Key. Â  Â  For more information on accessing the Configuration Editor and editing settings, refer to Answer ID 1960: Editing Configuration Settings."
test_sim_incidents(i1, i2)

['enable customize support home page']
Cluster:  136
['customer portal end user page home.php file define functionality support home page staff profile allow access customer portal studio edit customize home.php file include add remove widget text feature file path access home.php file euf development view page layout page define stylesheets use page access view template directory disable support home page page remove reference home.php file file template addition set default home page page page remove reference home page default file include tab home page navigation bar default button page define rn_standard.php rn_standard_account.php template edit template remove reference home.php file customize page link home page edit file good remove link home.php file set default end user page page set cp_home_url configuration set page widget include redirect function home page set default home page file user direct page define example set cp_home_url set answer list user log direct answer lis

In [ ]:
i1 = "Can I export reports from Oracle B2C Service into another format for analysis?"
i2 = "It is possible to export report information from the Oracle B2C Service application into a format that can be used in other analysis software. Data from both custom reports and standard reports can be exported. You are able Â to export more than the first page up to 100,000 rows, however, this ability is not available if the report is queued. You can only export one level of data at a time. That is, if your report includes multiple drill-down levels, you will only export data for the level that you are viewing. The export function does not automatically export all data for all levels and options in the report.    To export data from a report, Â use the steps below:  Generate the report of interest.   From the tool bar, click the Report tab.   Click the Export button.   Select the appropriate format: Â HTML, PDF, Excel, Â XML, Image, Clipboard Â or Delimited. Â   Â  Note: Â  In order to export to Excel, you must have Excel 2003 or later releases, Â due to a change in design that takes advantage of XML parsing that is available in Excel 2003 or later releases.   When you select an option, the Export Options window allows you to specify where the exported file is saved and the opportunity to launch the application associated with that file.  Notes: Â  Â  Ã¢â‚¬Â¢ Â  Exporting to HTML: When you select HTML, selecting relative formatting will create a report that will display correctly most often due to browser support, while absolute formatting will be more precise when the viewer uses a browser that supports that type of formatting. Ã¢â‚¬Â¢ Â  Exporing to Excel: When exporting in this format, the version of Microsoft Excel installed on your workstation is automatically checked and the report is exported as an .xls or .xlsx file depending on the version. When exporting a report with text areas to Excel, it is difficult to determine where to place the text fields. This is because the Oracle B2C Service Analytics engine records the location of the text area with absolute positioning (for example, we might place the upper left corner of a text field at the 13th pixel from the left and the 42nd pixel from the top) however Excel will only allow us to specify which cell in the grid we should place the text in. Therefore, although we do our best to try and place the text appropriately, it may not be placed in the same position that it was in the report. If you would like to see the text areas exactly as you have entered them in the report, you must export the report to a format Â  Â that supports absolute positioning (such as HTML).  For additional information, refer to the 'Export report and dashboard data' section in online documentation for the version your site is currently running. To access Oracle B2C Service manuals and documentation online, refer to the Documentation for Oracle B2C Service Products."
test_sim_incidents(i1, i2)

['export report oracle b2c service format analysis']
Cluster:  71
['possible export report information oracle b2c service application format use analysis software datum custom report standard report export able export page 100,000 row ability available report queue export level datum time report include multiple drill level export datum level view export function doe automatically export datum level option report export datum report use step generate report interest tool bar click report tab click export button select appropriate format html pdf excel xml image clipboard delimit note order export excel excel 2003 late release change design advantage xml parse available excel 2003 late release select option export option window allow specify export file save opportunity launch application associate file note ã¢â‚¬â¢ export html select html select relative format create report display correctly browser support absolute format precise viewer use browser support type format ã¢â‚¬â¢ exporin

## Max Anzahl von KB Artikel in einem Cluster

In [ ]:
dups_cluster = cluster_view.pivot_table(index=['Cluster'], aggfunc='size')
print('Max of kb articles in one cluster: ', dups_cluster.max())
dups_cluster

Max of kb articles in one cluster:  34


Cluster
0       7
1       3
2       4
3      30
4       4
       ..
245     6
246     2
247     2
248     1
249     1
Length: 250, dtype: int64

**Visulaisierung**

In [ ]:
#import csv

#distortions = []
#max = 50
#K = range(1,max) # for max of 250 Clusters

#for k in tqdm(K):
  #kmeanModel = KMeans(n_clusters=k, init='k-means++', max_iter=1000, n_init=1)
  #kmeanModel.fit(resolution_vec) # x is defined in "Ansatz: Text Clustering"
  #distortions.append(kmeanModel.inertia_)

#df = pd.DataFrame(distortions)
#df.to_csv('distortions_elbow_method_' + str(max) + '.csv', index=False)

In [ ]:
#import matplotlib.pyplot as plt

#dis = pd.read_csv('distortions_elbow_method_' + str(max) + '.csv', sep=';')

# Create a diagram
#plt.figure(figsize=(16,8))
#plt.plot(K, dis, 'bx-')
#plt.xlabel('k')
#plt.ylabel('Distortion')
#plt.title('The Elbow Method showing the optimal k')
#plt.show()

# Neuer Abschnitt

## Vorhersage

In [ ]:
import pickle


with open('documents.pickle', 'wb') as handle:
    pickle.dump(documents, handle)

with open('vectorizer.pickle', 'wb') as handle:
    pickle.dump(vectorizer, handle)

with open('model.pickle', 'wb') as handle:
    pickle.dump(model, handle)
with open('cluster_view.pickle', 'wb') as handle:
    pickle.dump(cluster_view, handle)
with open('nlp.pickle', 'wb') as handle:
    pickle.dump(nlp, handle)
with open('cleankb_resolution.pickle', 'wb') as handle:
    pickle.dump(cleankb_resolution, handle)


In [ ]:
from google.colab import files
files.download('vectorizer.pickle')
files.download('documents.pickle')
files.download('model.pickle')
files.download('cleankb_resolution.pickle')
files.download('cluster_view.pickle')
files.download('nlp.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#data = pd.read_csv('cluster_results_1.csv')
#data.columns
data = cluster_view
data
#data.set_index('Unnamed: 0')
#data
#cluster_view

,Text,Cluster
1,"[incident, assign, notification, incident, ass...",32
3,"[rule, feature, oracle, service, allow, assign...",108
5,"[send, response, contact, email, address, fiel...",8
7,"[proper, configuration, maintenance, environme...",72
9,"[page, oracle, service, file, attach, associat...",12
...,...,...
3139,"[screen, activate, list, suggest, resource, fo...",204
3141,"[submit, equipment, inventory, relate, request...",204
3143,"[forward, incident, system, automatically, set...",47
3145,"[yes, long, url, pass, setup, allow, http, nat...",101


In [ ]:
import math

def predict_cluster(incident):
  try:
    #data = pd.read_csv('cluster_results_1.csv')
    data = cluster_view
    # Identify the index of the question
    index = 0
    index_cluster = []

    for i in data:
      if i == incident[0]:
        index_cluster.append(index)
      index += 1
  
    inc = nlp(incident[0])
    #print(inc)
    # Predict a cluster
    incidentvec = vectorizer.transform(incident)
    prediction1 = model.predict(incidentvec)
  
    # print results
    print('Incident: ', incident[0]) # String
    print('Cluster: ', prediction1) # Cluster of Incident



    #print(data)
    cluster = []
    for i in data.index.tolist():
      if data.Cluster.loc[i] == prediction1:
        cluster.append(i)
    #print(data.index)
    cluster = []
    
    
  
    
    #print(cluster_view)
    #print(kb.index.tolist())
    #print(data)
    for i in data.index.tolist():
      if data.Cluster.loc[i] == prediction1:
        cluster.append(i)

    if len(index_cluster) > 0:  # Cluster of his resolution if exist
      print('Cluster of Resolution: ', data.Cluster[index_cluster[0]])

   ## Ab hier: Clustergröße reduzieren durch Subcluster

    min_cluster = 7
    size = len(cluster)

    while size > min_cluster:
      clean_resu_mini = cleankb_resolution.copy()
      documents_mini = []
      for i in cluster:
        documents_mini.append(" ".join(clean_resu_mini[i]))


      df_documents_mini = pd.DataFrame({
          'Resolution': documents_mini,
          'Index': cluster})
      

      documents_mini = df_documents_mini['Resolution'] 

      vectorizer_mini = TfidfVectorizer(stop_words='english')
      X_mini = vectorizer_mini.fit_transform(documents_mini)
      incidentvec_min = vectorizer_mini.transform(incident)

      minimized_k = int(math.ceil(size/min_cluster))

      model_mini = KMeans(n_clusters=minimized_k, init='k-means++', max_iter=1000, n_init=1)
      model_mini.fit(X_mini)

      # Predict a cluster
      prediction_min = model_mini.predict(incidentvec_min)
    
      # print result
      #print('smaller Cluster: ', prediction_min) # Cluster of Incident in smaller Cluster

      cluster_results_min = {'Resolution': [],
                  'Cluster': []
                  }
      #print(df_documents_mini['Resolut'])
      for i in df_documents_mini['Resolution']:
        Y = vectorizer_mini.transform([i])
        prediction = model_mini.predict(Y)
        cluster_results_min['Resolution'].append(i)
        cluster_results_min['Cluster'].append(int(prediction))

      cluster_view_min = pd.DataFrame(cluster_results_min)
      #cluster_view_min.to_csv('cluster_results_min.csv', index=False)

      data = cluster_view_min
  
      cluster = []
      for i in data.index:
        if data.Cluster.iloc[i] == prediction_min:
          cluster.append(df_documents_mini['Index'][i])  

      
      size = len(cluster)

    # Cluster reduzieren ist beendet



    #if len(index_cluster) > 0:  # Cluster of his resolution if exist
     # print('Cluster of Resolution: ', data.Cluster[index_cluster[0]])

    return cluster

  except ValueError as err:
    print(err)

In [ ]:
print(predict_cluster(["Using HTML in an answer in 18A and newer: How do I enter HTML into an answer so that it renders when viewed from the end-user pages?"]))

Incident:  Using HTML in an answer in 18A and newer: How do I enter HTML into an answer so that it renders when viewed from the end-user pages?
Cluster:  [157]
[13, 950, 2515, 2899, 2927, 3031]


In [ ]:
print(predict_cluster(["""Oracle B2C Service Configuration Assistant on Oracle Cloud Portal.
Rescheduling or cancelling an update cutover.
Documentation for Oracle B2C Service Products.
Requesting an Oracle B2C Service Update.
Auto Update Programm.
How do I change my Personal Data.
How do i configure my account.
change login security. """]))

Incident:  Oracle B2C Service Configuration Assistant on Oracle Cloud Portal.
Rescheduling or cancelling an update cutover.
Documentation for Oracle B2C Service Products.
Requesting an Oracle B2C Service Update.
Auto Update Programm.
How do I change my Personal Data.
How do i configure my account.
change login security. 
Cluster:  [134]
[680, 1334, 1630, 1788, 1966, 2545]


In [ ]:
def print_kb_article(index):
  #kb = pd.read_csv('answers_v9.csv', sep=';')
  #kb = kb.dropna(how='any')
  resolution = []
  if isinstance(index, list):
    for i in index:
      resolution.append([i, 'https://cx.rightnow.com/app/answers/detail/a_id/' + str(i), kb.Resolution.loc[i]])
  return resolution

In [ ]:
temp = print_kb_article(predict_cluster(["Can I receive an email when users have requested help and an incident is created?"]))
#temp = predict_cluster(["Can I receive an email when users have requested help and an incident is created?"])
for i in temp:
  print(i)

Incident:  Can I receive an email when users have requested help and an incident is created?
Cluster:  [47]
[41, 'https://cx.rightnow.com/app/answers/detail/a_id/41', '  In Oracle B2C Service, you can either manually forward a specific incident of interest to non-staff members or you can configure a workflow rule to automatically forward an incident to a specific email address that you specify. Enable Appropriate Messages Â\xa0 In order to manually forward an incident, the Admin Forward Incident Â\xa0message template Â\xa0must be enabled. To have a rule forward a copy of the incident, the E-mail Incident Information message template must be enabled. Messages and notifications can be enabled or disabled under Configuration &gt; Site Configuration &gt; Message Templates &gt; select the interface you wish to edit messages for. Next, select Administrator Emails &gt; Incident. Confirm the template type you want is enabled to Send and enabled as HTML (if desired).  Manually Forwarding Incide

# Show Cases

In [ ]:
temp = print_kb_article(predict_cluster(["How do I schedule my site to be updated?"]))

for i in temp:
  print(i)

In [ ]:
temp = print_kb_article(predict_cluster(["How do I change my Personal data"]))

for i in temp:
  print(i)

In [ ]:
temp = print_kb_article(predict_cluster(["How much files can i attache to an incident. Is there a limit of the numbers of files or maximum size?"]))
# Index 9

for i in temp:
  print(i)

In [ ]:
temp = print_kb_article(predict_cluster(["How i can reset or change my user password"]))
# Index 3137

for i in temp:
  print(i)

# Mit besuchten Webseiten

In [ ]:
import pandas as pd

# Einlesen und Verarbeiten der CSV-Datei
case_csv =  pd.read_csv('showcase.csv', sep=';', encoding="cp1252")
case_csv

In [ ]:

def predict_visited_websites(file):

  # Einlesen und Verarbeiten der CSV-Datei
  case_csv =  pd.read_csv('' + file + '.csv', sep=';', encoding="cp1252")
  
  # Die letzte Zeile soll das Ergebnis darstellen
  test = []
  result = []

  # Ermittlung des letzten Eintrags / mit max(x) gab es immer Probleme
  index = case_csv.index.tolist()
  maxi = 0
  for i in index:
    if i > maxi:
      maxi = i

  # Aufteilung in besuchte Seiten und Lösung (letzte Zeile)
  for i in case_csv.index:
    if i != maxi:
      test.append(case_csv.iloc[i])
    else:
      result.append(case_csv.iloc[i])


  # Topic, Question & Text werden zu einem String verbunden 
  # Anschließend wird darauf die Vorhersage getroffen
  temp = []
  str_to_pred = []
  predictions = []
  sammlung = []
  for z in range(len(test)):
      strings = []
      print(z)
      strings.append(test[z]['Topic'])
      strings.append(test[z]['Question'])
      strings.append(test[z]['Text'])
      #Notlösung sonst ist die Anzeige zerstört wegen Leerzeichen etc.
      string_pred = ' '.join(nlp(' '.join(strings))) 
    
      sammlung.append(predict_cluster([string_pred]))
      #print(predict_cluster([string_pred]))
      #sammlung = print_kb_article(predict_cluster([string_pred]))


  #print(sammlung)
  sam = []
  for y in sammlung:
      sam = sam + y
  #print(result[0]['ID'])
  res_id = result[0]['ID']
  print("\n")
  
  # EVLT
  if res_id in sam:
    print("Lösung ist in den vorgeschlagenen KB-Artikel")
  else:
    print('Lösung wurde nicht erkannt')

  # Element das am häufigsten vorkommt
  liste = []
 

  for i in range(len(sam)):
      liste.append(sam.count(sam[i]))
  
  maximum = 0
  for i in liste:
    if i > maximum:
      maximum = i

  maxi_url = []
  maxi = []
  for i in range(len(liste)):
      if sam.count(sam[i]) == maximum:
          maxi.append(sam[i])
          maxi_url.append("https://cx.rightnow.com/app/answers/detail/a_id/" + str(sam[i]))

  return maxi, maxi_url



# ***************************************************************************************************

# Output
max, maxUrl = predict_visited_websites('showcase')

print(max)
for i in maxUrl:
  print(i)




In [ ]:
def clean(z):
  nlpText = spacy.load("en_core_web_sm")
  #nlpText.add_pipe(lemmatizer,name='lemmatizer',after='ner')
  #nlpText.add_pipe(notNUM,name='notNUM',after='ner')
  #nlpText.add_pipe(single_char,name='single_char',after='ner')
  #nlpText.add_pipe(noSYM,name='noSYM',after='ner')
  #nlpText.add_pipe(rm_punct,name='rm_punct',after='ner')
  #nlpText.add_pipe(rm_adj,name='rm_adj',after='ner')
  #nlpText.add_pipe(rm_other,name='rm_other',after='ner')
  #nlp.add_pipe(rm_verb,name='rm_verb',after='ner')
  #nlpText.add_pipe(remove_stopwords, name="stopwords", last=True) 

  z = re.sub('ã*', '', z)
  z = re.sub('\n', '', z)
  z = re.sub('â*', '', z)
  z = re.sub('ã*', '', z)
  z = re.sub('ë*', '', z)
  z = re.sub('\d*', '', z)
  z = re.sub('¢*', '', z)

  ret = nlp(z)
  return ret 


# Clean 
#b = print_kb_article(max)
#for o in b:
#  print(clean(o))